In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('dermatology.csv')
df

In [ ]:
df.isna().sum()

In [ ]:
df.age.values

In [ ]:
from sklearn.impute import SimpleImputer
imp= SimpleImputer(missing_values=np.nan, strategy='most_frequent')
age_imp=pd.DataFrame(imp.fit_transform(np.array(df.age).reshape(-1,1)))

In [ ]:
df.age=age_imp

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
x=df.drop(columns=['class'])
y=df['class']

x.shape

In [ ]:
from sklearn.feature_selection import VarianceThreshold
select=VarianceThreshold(threshold=0)
x=select.fit_transform(x)

x.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y= le.fit_transform(y)

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=15)

In [ ]:
from sklearn.model_selection import GridSearchCV
PARAMETERS = {
              "max_depth":[ 5, 10],
              "min_child_weight":[2,10],
              "learning_rate":[0.01, 0.1],
              "n_estimators":[100,200]}
model_gs = GridSearchCV(xgb_model,param_grid=PARAMETERS,cv=3,scoring="accuracy")
model_gs.fit(xtrain,ytrain)

print(model_gs.best_params_)

In [ ]:
xgb_model = xgb.XGBClassifier(learning_rate= 0.1, max_depth= 10, min_child_weight=2,n_estimators= 100)
xgb_model.fit(xtrain, ytrain)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
ypred = xgb_model.predict(xtest)
print(accuracy_score(ytest, ypred))
print(classification_report(ytest, ypred))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(xtrain)
X_test_reduced = pca.transform(xtest)

In [ ]:
xgb_model.fit(X_train_reduced, ytrain)
ypred_pca= xgb_model.predict(X_test_reduced)
print(accuracy_score(ytest, ypred_pca))
print(classification_report(ytest, ypred_pca))

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
X_reduced_tsne = tsne.fit_transform(xtrain)

In [ ]:
X_reduced_tsne.shape

In [ ]:
plt.figure(figsize=(13,10))
plt.scatter(X_reduced_tsne[:, 0], X_reduced_tsne[:, 1], c=ytrain, cmap="jet")
plt.axis('off')
plt.colorbar()
plt.show()